<a href="https://colab.research.google.com/github/zbooster/Landmark-Image-Classification/blob/main/resnet50_transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1.데이터 준비하기
preprocessing.ipynb 에서 Resize한 데이터를 Google Drive에 업로드 하였으며,
이 과정에서는 Colab에 연결하여 Download하고 압축을 해제하는 작업을 수행한다.

In [6]:
!gdown 1_yGDePPvjlsTe4qi3X6LqY2DPEB97Itq
!unzip -o -qq '/content/resizeds_0.1x.zip' -d './datasets'

# 2.Weights & Biases 설정
하이퍼파라미터 튜닝을 위해 Weights & Biases에 로그인하여 데이터를 쌓고 Sweeps를 활용.

#### Package install

In [1]:
!pip install wandb -Uq

     |████████████████████████████████| 1.8 MB 18.0 MB/s 
     |████████████████████████████████| 181 kB 64.6 MB/s 
     |████████████████████████████████| 157 kB 51.3 MB/s 
     |████████████████████████████████| 63 kB 958 kB/s 
     |████████████████████████████████| 157 kB 58.2 MB/s 
     |████████████████████████████████| 157 kB 56.5 MB/s 
     |████████████████████████████████| 156 kB 55.4 MB/s 


#### Login

In [2]:
import wandb

wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 

··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

#### Sweep config

method: 모든 파라미터를 다 테스트하기 위해 grid를 선택했다.

In [7]:
sweep_config = {
    'method': 'grid'
}

metric: 검증(Validation) 데이터셋의 정확도(Accuracy)를 최대(maximize)로 올리는 것으로 선택했다.

In [8]:
metric = {
    'name': 'Validation Accuracy',
    'goal': 'maximize'   
    }

sweep_config['metric'] = metric

parameters_dict: 파라미터 변화에 따른 정확도를 살펴보기 위해 최대한 많이 설정했다.
- batch_size
- resize_pixel
- learning_rate
- step_size
- crop_scale : transform 과정에서 RandomCrop함수의 픽셀값을 Resize값의 몇 x배로 설정
- grad_false_idx : 기존에 이미지넷에서 학습한 값을 얼마나 재 활용할 것인지


In [9]:
parameters_dict = {
    'batch_size': {
        'values': [64, 128, 256] },
    'resize_pixel': {
        'values': [64, 128, 256] },
    'crop_scale': {
        'values': [0.6, 0.7, 0.8, 0.9] },
    'learning_rate': {
        'values': [0.01, 0.001, 0.0001]},
    'grad_false_idx': {
        'values': [5, 6, 7] },
    'step_size': {
        'values': [5, 6, 7] },
    'gamma': {
        'value': 0.1},
    'epochs': {
        'value': 25},
    'class_size': {
        'value': 84},
    }

sweep_config['parameters'] = parameters_dict

In [10]:
import pprint

pprint.pprint(sweep_config)

{'method': 'grid',
 'metric': {'goal': 'maximize', 'name': 'Validation Accuracy'},
 'parameters': {'batch_size': {'values': [64, 128, 256]},
                'class_size': {'value': 84},
                'crop_scale': {'values': [0.6, 0.7, 0.8, 0.9]},
                'epochs': {'value': 25},
                'gamma': {'value': 0.1},
                'grad_false_idx': {'values': [5, 6, 7]},
                'learning_rate': {'values': [0.01, 0.001, 0.0001]},
                'resize_pixel': {'values': [64, 128, 256]},
                'step_size': {'values': [5, 6, 7]}}}


#### Initialize the Sweep

In [11]:
sweep_id = wandb.sweep(sweep_config, project="Resnet50_Transfer_Learning")

Create sweep with ID: x8qjqrcx
Sweep URL: https://wandb.ai/zbooster/Resnet50_Transfer_Learning/sweeps/x8qjqrcx


# 3.데이터셋(Dataset) 준비하기


## 3.1.정규화(Normalization)
이미지 픽셀의 평균, 표준편차를 계산하여 정규화하기 위한 함수이다.

In [12]:
import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder

def get_mean_and_std(dataloader):
    channels_sum, channels_squared_sum, num_batches = 0, 0, 0
    for data, _ in dataloader:
        # Mean over batch, height and width, but not over the channels
        channels_sum += torch.mean(data, dim=[0,2,3])
        channels_squared_sum += torch.mean(data**2, dim=[0,2,3])
        num_batches += 1
    
    mean = channels_sum / num_batches

    # std = sqrt(E[X^2] - (E[X])^2)
    std = (channels_squared_sum / num_batches - mean ** 2) ** 0.5

    return mean, std

def build_nomalize_loader(set_type, resize_pixel):
    transform = transforms.Compose([
        transforms.Resize([resize_pixel, resize_pixel]),
        transforms.ToTensor()
        ])

    dataset = ImageFolder(root='/content/datasets/resizeds/' + set_type, transform=transform)
    loader = DataLoader(dataset=dataset, batch_size=64)

    return loader

파라미터를 바꿔가며 테스트할 때마다 값을 구할 필요는 없다. 미리 계산해놓고 dict형태로 저장한다.

In [25]:
from collections import defaultdict

n_mean = defaultdict(dict)
n_std  = defaultdict(dict)

for rpixel in [ 64, 128, 256 ]:
  n_mean[rpixel] = defaultdict(list)
  n_std[rpixel] = defaultdict(list)
  for dtype in ['Training', 'Validation']:
    mean, std = get_mean_and_std(build_nomalize_loader(dtype, rpixel))
    n_mean[rpixel][dtype].extend(list(mean.numpy()))
    n_std[rpixel][dtype].extend(list(mean.numpy()))

n_mean, n_std

(defaultdict(dict,
             {64: defaultdict(list,
                          {'Training': [0.49465266, 0.49119774, 0.49918783],
                           'Validation': [0.4859542, 0.4816538, 0.49004397]}),
              128: defaultdict(list,
                          {'Training': [0.4946506, 0.49119437, 0.49918264],
                           'Validation': [0.4859544, 0.48165187, 0.49004138]}),
              256: defaultdict(list,
                          {'Training': [0.4946475, 0.49119085, 0.49917814],
                           'Validation': [0.48595107,
                            0.48164916,
                            0.49003717]})}),
 defaultdict(dict,
             {64: defaultdict(list,
                          {'Training': [0.49465266, 0.49119774, 0.49918783],
                           'Validation': [0.4859542, 0.4816538, 0.49004397]}),
              128: defaultdict(list,
                          {'Training': [0.4946506, 0.49119437, 0.49918264],
                    

## 3.2.데이터로더(DataLoader)
* transforms
  * resize -> 상하/좌우 반전(Train phase만) -> RandomCrop -> 정규화
* ImageFolder에 사용하기 적합한 Directory 구조로 만들어져 있으므로 ImageFolder로 데이터셋을 만든다.
* Colab의 cpu core는 2개가 일반적이므로 num_workers는 2로 설정하고 shuffle한다.
  


In [27]:
def build_dataset(batch_size, set_type, resize_pixel, crop_scale, n_mean, n_std):
  
    compose_list = []
    compose_list.append(transforms.Resize([resize_pixel, resize_pixel]))
    if set_type == 'Training':
      compose_list.append(transforms.RandomHorizontalFlip())
      compose_list.append(transforms.RandomVerticalFlip())
    compose_list.append(transforms.RandomCrop(int(resize_pixel * crop_scale)))
    compose_list.append(transforms.ToTensor())
    compose_list.append(transforms.Normalize(n_mean, n_std))
    
    transform = transforms.Compose(compose_list)
  
    data_dir = '/content/datasets/resizeds'
    dataset = ImageFolder(root=os.path.join(data_dir, set_type), transform=transform)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=2)
  
    return loader, len(dataset)

# 4.모델
* Resnet50 전이학습을 사용할 예정이므로 torchvision에서 Resnet50과 Weights를 가져와서 적용한다.
* device는 GPU를 사용한다.

In [28]:
from torchvision.models import resnet50, ResNet50_Weights
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def build_network(class_size):
    network = resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)
    num_ftrs = network.fc.in_features
    network.fc = nn.Linear(num_ftrs, class_size)
    return network.to(device)

# 5.옵티마이저와 스케쥴러
* 옵티마이저는 Adam을 사용한다.
* 기존 Resnet50의 Weight를 얼마나 재 사용할지 정한다.

In [31]:
import torch.optim as optim

def build_optimizer(network, learning_rate, grad_false_idx):
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, network.parameters()), 
                           lr=learning_rate)
    for idx, child in enumerate(network.children()):
      if idx < grad_false_idx:
        for param in child.parameters():
          param.requires_grad = False
    return optimizer

* 스케쥴러는 StepLR을 step_size와 gamma를 변경해가며 테스트할 수 있도록 작성했다.


In [30]:
from torch.optim import lr_scheduler

def build_scheduler(optimizer, step_size, gamma):
    exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)
    return exp_lr_scheduler

# 6.학습/검증 단계(Epoch)
* Loss 함수는 CrossEntropyLoss()를 설정합니다.

In [29]:
critetion = nn.CrossEntropyLoss()

## 6.1.학습(Train) 단계

In [32]:
def train_epoch(network, loader, datasize, optimizer, scheduler):
    network.train()

    running_loss = 0.0
    running_corrects = 0

    for _, (data, target) in enumerate(loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()

        with torch.set_grad_enabled(True):
          # ➡ Forward pass
          outputs = network(data)
          _, preds = torch.max(outputs, 1)
          loss = critetion(outputs, target)

          # ⬅ Backward pass + weight update
          loss.backward()
          optimizer.step()

        running_loss += loss.item() * data.size(0)
        running_corrects += torch.sum(preds == target.data)

    scheduler.step()

    epoch_loss = running_loss/datasize
    epoch_acc = running_corrects.double()/datasize

    return epoch_loss, epoch_acc

## 6.2.검증(Validation) 단계

In [33]:
def validation_epoch(network, loader, datasize, optimizer):
    network.eval()
    
    running_loss = 0.0
    running_corrects = 0

    for _, (data, target) in enumerate(loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()

        # ➡ Forward pass
        outputs = network(data)
        _, preds = torch.max(outputs, 1)
        loss = critetion(outputs, target)

        running_loss += loss.item() * data.size(0)
        running_corrects += torch.sum(preds == target.data)

    epoch_loss = running_loss/datasize
    epoch_acc = running_corrects.double()/datasize

    return epoch_loss, epoch_acc

# 7.수행

In [34]:
import time

def train(config=None):
    # Initialize a new wandb run
    with wandb.init(config=config):
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config

        t_loader, t_datasize = build_dataset(config.batch_size, 'Training', 
                                             config.resize_pixel, config.crop_scale
                                             , n_mean[config.resize_pixel]['Training']
                                             , n_std[config.resize_pixel]['Training']
                                             )
        
        v_loader, v_datasize = build_dataset(config.batch_size, 'Validation', 
                                             config.resize_pixel, config.crop_scale
                                             , n_mean[config.resize_pixel]['Validation']
                                             , n_std[config.resize_pixel]['Validation']
                                             )
        
        network = build_network(config.class_size)
        optimizer = build_optimizer(network, config.learning_rate, config.grad_false_idx)
        scheduler = build_scheduler(optimizer, config.step_size, config.gamma)

        for epoch in range(config.epochs):
            since = time.time()
            
            epoch_loss, epoch_acc = train_epoch(network, t_loader, t_datasize, optimizer, scheduler)
            wandb.log({"Train Loss": epoch_loss, "Train Accuracy": epoch_acc * 100, "Epoch": epoch})

            epoch_loss, epoch_acc = validation_epoch(network, v_loader, v_datasize, optimizer)
            wandb.log({"Validation Loss": epoch_loss, "Validation Accuracy": epoch_acc * 100, "Epoch": epoch})

            time_elapsed = time.time() - since
            wandb.log({"Time Elapsed(s)": time_elapsed, "Epoch": epoch})


            

* Sweeps 에이전트가 파라미터 값을 바꿔가면서 반복 수행함. 결과는 저장

In [ ]:
wandb.agent(sweep_id, train)

wandb: Agent Starting Run: mx88xqdl with config:
wandb: 	batch_size: 64
wandb: 	class_size: 84
wandb: 	crop_scale: 0.6
wandb: 	epochs: 25
wandb: 	gamma: 0.1
wandb: 	grad_false_idx: 5
wandb: 	learning_rate: 0.01
wandb: 	resize_pixel: 64
wandb: 	step_size: 5
wandb: Currently logged in as: zbooster. Use `wandb login --relogin` to force relogin


Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]